You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Filter Operation:
o	Perform a "filter" operation on the retrieved launch data based on specific criteria.
o	Filter Logic: Filter the list of launches based on launch year and launch success status. Only include launches that match the specified criteria.
o	The result should be a list of dictionaries, containing only the launch records that satisfy the filter conditions.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URL and the filtering criteria, making them easily modifiable. Use parameters like: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
	FILTER_YEAR = 2019 (Set to an integer year, or None to disable year filtering)
	FILTER_SUCCESS = True (Set to True, False, or None to disable success filtering)
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, filtering criteria being applied, number of launches before filtering, number of launches after filtering, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + filtering). Print this time after the filtering operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting filtered list from the filter operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting filtered JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# Databricks Notebook
# SpaceX Launch Data Pipeline

# -----------------------
# Imports
# -----------------------
import requests
import json
import time
import logging

# -----------------------
# Control Parameters
# -----------------------
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
FILTER_YEAR = 2019          # Set to an integer (e.g., 2019) or None
FILTER_SUCCESS = True       # Set to True, False, or None

UPLOAD_URL = "https://httpbin.org/post"

# -----------------------
# Logging Configuration
# -----------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

# -----------------------
# Helper Functions
# -----------------------

def retrieve_spacex_launches(api_url):
    """Retrieve launch data from SpaceX API."""
    logger.info("Starting data retrieval from SpaceX API...")
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()
        launches = response.json()
        logger.info(f"Successfully retrieved {len(launches)} launches.")
        return launches
    except requests.exceptions.RequestException as e:
        logger.error(f"Error retrieving data from SpaceX API: {e}")
        raise

def filter_launches(launches, year=None, success=None):
    """Filter launches based on year and success status."""
    logger.info("Starting filtering operation...")
    logger.info(f"Filtering criteria - Year: {year}, Success: {success}")
    initial_count = len(launches)
    
    filtered = []
    for launch in launches:
        launch_year = int(launch.get('launch_year', 0))
        launch_success = launch.get('launch_success')

        if year is not None and launch_year != year:
            continue
        if success is not None and launch_success != success:
            continue
        filtered.append(launch)

    logger.info(f"Launches before filtering: {initial_count}")
    logger.info(f"Launches after filtering: {len(filtered)}")
    return filtered

def upload_results(upload_url, data):
    """Upload filtered results to httpbin.org."""
    logger.info("Starting upload to httpbin.org...")
    try:
        headers = {'Content-Type': 'application/json'}
        response = requests.post(upload_url, headers=headers, data=json.dumps(data), timeout=10)
        if response.status_code == 200:
            logger.info("Upload successful.")
        else:
            logger.error(f"Upload failed with status code {response.status_code}")
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error uploading data: {e}")
        raise

# -----------------------
# Main Execution
# -----------------------
if __name__ == "__main__":
    total_start_time = time.time()

    # Step 1: Data Retrieval
    try:
        main_start_time = time.time()
        launches = retrieve_spacex_launches(API_ENDPOINT_URL)

        # Step 2: Filtering
        filtered_launches = filter_launches(launches, FILTER_YEAR, FILTER_SUCCESS)
        main_end_time = time.time()
        main_execution_time = main_end_time - main_start_time
        logger.info(f"Main operation (retrieval + filtering) took {main_execution_time:.2f} seconds.")

        # Step 3: Upload Filtered Results
        upload_results(UPLOAD_URL, filtered_launches)

    except Exception as e:
        logger.error(f"Pipeline failed: {e}")

    total_end_time = time.time()
    total_execution_time = total_end_time - total_start_time
    logger.info(f"Total pipeline execution time: {total_execution_time:.2f} seconds.")
